In [1]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
ratings = pd.read_csv('ratings.dat', sep='::', header=None, engine='python',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('movies.dat', sep='::', header=None, engine='python',
                     names=['movie_id', 'title', 'genres'], encoding='latin-1')

In [4]:
user_item_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

In [5]:
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

In [6]:
def get_movie_id(movie_title):
    try:
        movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movie_id'].values[0]
        return movie_id
    except IndexError:
        return None

In [7]:
def get_recommendations(movie_title, num_recommendations=5):
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the database."

    similar_movies = item_similarity_df[movie_id].sort_values(ascending=False).index[1:num_recommendations+1]
    recommended_titles = movies[movies['movie_id'].isin(similar_movies)]['title'].values
    return recommended_titles

def get_movie_titles(movie_ids):
    return movies[movies['movie_id'].isin(movie_ids)]['title'].values


In [8]:
def chatbot_response(user_input):
    user_input = user_input.lower()

    # Define patterns and corresponding responses
    patterns = {
        r'hello|hi': "Hello! How can I help you today?",
        r'how are you': "I'm just a bot, but I'm doing great! How about you?",
        r'what is your name|who are you': "I'm a simple chatbot created to assist you.",
        r'help|can you recommendation a movie|movie': "Sure, I'm here to help! You can ask me for movie recommendations by telling me a movie you like. For example, 'I like Titanic'.",
        r'bye|goodbye': "Goodbye! Have a great day!"
    }

    for pattern, response in patterns.items():
        if re.search(pattern, user_input):
            return response
    
    # Check if the user is asking for recommendations
    if re.search(r'i like .+', user_input):
        movie_title = re.search(r'i like (.+)', user_input).group(1)
        recommendations = get_recommendations(movie_title)
        if isinstance(recommendations, str):
            return recommendations
        if recommendations.size > 0:
            rec_movies = ',\n '.join(recommendations)
            return f"If you like '{movie_title}', you might also enjoy:\n {rec_movies}."
        else:
            return "I couldn't find any recommendations for this movie."
    
    # Default response if no pattern matches
    return "I'm sorry, I didn't understand that. Can you please rephrase?"

In [10]:
def main():
    print("Chatbot: Hello! I am here to recommend a movie type <i like movie_name> Type 'bye' to exit.")
    while True:
        user_input = input("You: ")
        if re.search(r'bye|goodbye', user_input.lower()):
            print("Chatbot: Goodbye! Have a great day!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    main()

Chatbot: Hello! I am here to recommend a movie type <i like movie_name> Type 'bye' to exit.
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: I'm sorry, I didn't understand that. Can you pl